# Búsqueda de codones de inicio en un archivo FASTA

In [1]:
from Bio import SeqIO
from Bio.Data import CodonTable

In [2]:
def buscar(secuencia,codon):
    """(seq,str) -> list
    Dada una secuencia y un codon
    Retorna la lista de indices en los cuales comienza ese codon en la secuencia
    
    >>> buscar("ATGCATG","ATG")
    [0, 4]
    """
    lista = []
    indice = 0
    while indice < len(secuencia):
        pos = secuencia.find(codon,indice)
        if pos == -1 :
            break
        indice = pos + 1
        lista.append(pos)
    return( lista )

In [3]:
buscar("ATGCATG","ATG")

[0, 4]

In [4]:
def buscar_starts(archivo, id=11):
    """(str,int) -> (list,list)
    Dado un archivo FASTA, con un record, busca las posiciones de comienzo de los codones de start
    Retorna dos listas:
    La primera con las posiciones en la que inician los codones en la cadena sentido
    La segunda con las posiciones en que inician los codones en la cadena antisentido 
    (segun la numeracion de la cadena sentido)
    WARNING: Si hay mas de un record en el archivo, los resultados de las multiples cadenas se mezclaran
    """
    # Inicializo las listas
    sentido = []
    antisentido = []
    # Obtengo la tabla de codones
    tabla = CodonTable.unambiguous_dna_by_id[ id ]
    # Ejecuto para cada record del archivo
    for seqrec in SeqIO.parse( archivo ,"fasta"):
        # Obtengo la secuencia reversa complementaria
        rc = seqrec.seq.reverse_complement()
        # Ejecuto la busqueda para cada codon de inicio de la tabla de codones
        for codon in tabla.start_codons:
            # Busco los codones con la funcion definida anteriormente
            sentido.extend( buscar(seqrec.seq, codon) )
            antisentido.extend( buscar(rc, codon) )
    # list-comprehensions para pasar las posiciones de la reversa complementaria a la sentido
    return( sentido, [ ( len(rc) - 1 ) - pos for pos in antisentido ] )
        

Para más información en [list comprehensions](https://docs.python.org/2/tutorial/datastructures.html#list-comprehensions)

In [5]:
posiciones = buscar_starts("sequence.fasta")
posiciones[1][:10]

[1664910,
 1664821,
 1664799,
 1664789,
 1664774,
 1664737,
 1664713,
 1664664,
 1664638,
 1664605]

# Conteo de k-mers en un archivo FASTA

Tambien pueden ver la implementación de [Python for Biologists](http://pythonforbiologists.com/index.php/kmer-counting-business-card/)

In [6]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
seq = Seq("ATGCATG", IUPAC.unambiguous_dna)

In [7]:
def kmer_count(archivo,k=3):
    """(str,int) -> dict
    Dado un archivo FASTA genera un diccionario con la frecuencia de k-mers
    """
    contador = {}
    for seqrec in SeqIO.parse( archivo ,"fasta"):
        seq = str( seqrec.seq )
        for pos in range(0,len(seq)-k+1):
            sub = seq[pos:pos+k]
            if sub in contador:
                contador[ sub ] += 1
            else:
                contador[ sub ]  = 1
    return( contador )
    

In [8]:
contador = kmer_count("sequence.fasta")

In [9]:
contador.items()[:10]

[('ACC', 12731),
 ('NAA', 1),
 ('CCY', 1),
 ('AGG', 19393),
 ('CCT', 18757),
 ('AGC', 18862),
 ('AGA', 39295),
 ('GSG', 1),
 ('CCG', 2807),
 ('AGT', 23069)]

# Guardando los Resultados en Archivos de Texto

Las posiciones están almacenadas en un objeto de tipo *tuple* con **dos elementos**, los cuales corresponden a dos objetos de tipo *list* que contienen valores del tipo *int*. La primera lista es la cadena **sentido**, mientras que la segunda es la **antisentido**.

In [10]:
print( type(posiciones) )
print( type(posiciones[0]) )
print( type(posiciones[0][0]) )

<type 'tuple'>
<type 'list'>
<type 'int'>


Guardaremos esto en un sólo archivo, en al carpeta de trabajo, que tenga por nombre **starts.txt**
El archivo contendrá dos columnas. La primera indicando si es una posición de la cadena *sentido* o *antisentido*. La segunda columna, separando con *tab*, tendrá la posición del codón de start.

Primero [abrimos el archivo](https://docs.python.org/2/tutorial/inputoutput.html#reading-and-writing-files) en modo escritura (**```w```**) usando la función **```open()```**

In [11]:
archivo = open("starts.txt", "w")

In [12]:
type( archivo )

file

In [13]:
print( archivo )

<open file 'starts.txt', mode 'w' at 0x7f74f0286660>


Ahora vamos a iterar sobre cada una de las listas. Los índices dentro de la tupla son ```0``` para la lista de la cadena sentido, ```1``` para la antisentido. Esas listas las podemos recorrer completas con un **```for```**. Para guardar una línea de texto en el archivo que abrimos usaremos el método **```write()```** de la clase **```file```**. La cadena **```str```** que queremos imprimir la generamos sumando pequeñas cadenas. Usaremos **```\t```** para indicar un tab y **```\n```** para indicar nueva línea.

In [14]:
for pos in posiciones[0]:
    archivo.write( "sentido\t" + str(pos) + "\n" )

In [15]:
for pos in posiciones[1]:
    archivo.write( "antisentido\t" + str(pos) + "\n" )

Una vez guardadas las líneas, tenemos que cerrar el archivo usando el método **```close```** para que el contenido se guarde.

In [16]:
archivo.close()

Estando en Ubuntu, es fácil ejecutar un pequeño **comando de la terminal** desde IPython, sólo hay que escribir **```!```** antes de escribir el comando que usamos en la terminal. Por ejemplo podemos usar **```head```** y **```tail```** para ver si nuestro archivo esta correcto.

In [17]:
! head -2 starts.txt

sentido	19
sentido	63


In [18]:
! tail -2 starts.txt

antisentido	841
antisentido	709


Una cosa útil que podemos chequear es si la cantidad de líneas del archivo es la deseada:

In [19]:
len(posiciones[0]) + len(posiciones[1])

493438

In [20]:
! wc -l starts.txt

493438 starts.txt


#### Guardando el contenido del diccionario de codones

Otra cosa útil sería guardar el diccionario de codones, también en un formato de dos columnas separadas por tabs. Pero de manera de sólo incluir el conteo de los codones que no contienen ambigüedades de IUPAC. Podemos usar conjuntos (**```set```**)

In [21]:
set(IUPAC.ambiguous_dna.letters)

{'A', 'B', 'C', 'D', 'G', 'H', 'K', 'M', 'N', 'R', 'S', 'T', 'V', 'W', 'Y'}

In [22]:
set(IUPAC.unambiguous_dna.letters)

{'A', 'C', 'G', 'T'}

In [23]:
ambiguas = set.difference( set(IUPAC.ambiguous_dna.letters), set(IUPAC.unambiguous_dna.letters) )

Los diccionarios tiene maneras especiales de ser iteradas. Un de ellas es usar el método **```iteritems()```** que retorna en cada vuelta del *loop* **```for```** una tupla conteniendo una *key* y un *value* del diccionario.

In [24]:
contador.iteritems()

<dictionary-itemiterator at 0x7f74f02ad2b8>

In [25]:
for item in contador.iteritems():
    if len( ambiguas.intersection( set( item[0] ) ) ) == 0:
        print item
        break # Imprimo solo el primero, y salgo del loop, para evitar llenar la pantalla

('ACC', 12731)


Una manera de poder abrir un archivo, dejando que *Python* se encargue de cerrarlo cuando terminamos de usarlo, o si surge algún error, es usando la sentencia **```with```** junto a **```open```**. Es la manera recomendada para abrir archivos. Acá la vamos a usar para abrir el archivo **codones.txt** en modo escritura (**```w```**).

In [26]:
with open("codones.txt", "w") as archivo:
    for llave, valor in contador.iteritems():
        if len( ambiguas.intersection( set( llave ) ) ) == 0:
            archivo.write( llave + "\t" + str(valor) + "\n" )

In [27]:
! head codones.txt

ACC	12731
AGG	19393
CCT	18757
AGC	18862
AGA	39295
CCG	2807
AGT	23069
CCA	22867
CCC	10914
TAT	59194


In [28]:
! wc -l codones.txt

64 codones.txt
